# Imports

In [176]:
import numpy as np
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from pandas.plotting import scatter_matrix
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import confusion_matrix, f1_score, average_precision_score, classification_report, fbeta_score, accuracy_score
from sklearn.feature_selection import RFECV

from statsmodels.stats.outliers_influence import variance_inflation_factor

import optuna
import statsmodels.api as sm
from boruta import BorutaPy

import custom_map

In [177]:
import importlib

importlib.reload(custom_map)

<module 'custom_map' from '/Users/dominikmika/PycharmProjects/Ridge-hillclimbing/custom_map.py'>

# Setup

In [178]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
bmi_median = data['bmi'].median()
data['bmi'] = data['bmi'].fillna(bmi_median)

target = "stroke"

categorical_raw = data.select_dtypes(include=['object']).columns.tolist()
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns.drop('id')

data = pd.get_dummies(
    data,
    columns=categorical_raw,
    drop_first=True,
    dtype=float
)

data = data.drop('id', axis=1)

binary_features = [
    'hypertension',
    'heart_disease',
    'stroke'
]

numerical_features = numerical_features.drop(binary_features)

numerical_binary_features = numerical_features.union(binary_features)

categorical_features = [
    col for col in data.columns
    if any(col.startswith(c + "_") for c in categorical_raw)
]

categorical_features = categorical_features + binary_features
categorical_features = list(dict.fromkeys(categorical_features))  # deduplikacja

print("Categorical:", categorical_features)
print("Numerical:", numerical_features.tolist())
print("Numerical + binary:", numerical_binary_features.tolist())

Categorical: ['gender_Male', 'gender_Other', 'ever_married_Yes', 'work_type_Never_worked', 'work_type_Private', 'work_type_Self-employed', 'work_type_children', 'Residence_type_Urban', 'smoking_status_formerly smoked', 'smoking_status_never smoked', 'smoking_status_smokes', 'hypertension', 'heart_disease', 'stroke']
Numerical: ['age', 'avg_glucose_level', 'bmi']
Numerical + binary: ['age', 'avg_glucose_level', 'bmi', 'heart_disease', 'hypertension', 'stroke']


In [179]:
X = data.drop(columns=[target])
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, shuffle=True)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [180]:
## hard coding because of stochasticity

all_features = ["age", "hypertension", "heart_disease", "avg_glucose_level",
                "bmi", "gender_Male", "gender_Other", "ever_married_Yes",
                "work_type_Never_worked", "work_type_Private", "work_type_Self-employed",
                "work_type_children", "Residence_type_Urban",
                "smoking_status_formerly smoked", "smoking_status_never smoked",
                "smoking_status_smokes"]

boruta_features = ["age", "avg_glucose_level", "bmi"]

corr_features = ['age', 'heart_disease', 'avg_glucose_level', 'hypertension', 'ever_married_Yes',
                 'smoking_status_formerly smoked', 'work_type_Self-employed', 'bmi']

mi_features = ["age", "hypertension", "gender_Other",
               "work_type_Private", "smoking_status_formerly smoked"]

rfe_features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'gender_Male', 'ever_married_Yes',
                'work_type_Never_worked', 'work_type_Private', 'work_type_Self-employed', 'work_type_children',
                'Residence_type_Urban', 'smoking_status_formerly smoked', 'smoking_status_never smoked', 'smoking_status_smokes',
                'bmi']

FEATURE_SETS = {
    "all": all_features,
    "boruta": boruta_features,
    "correlation": corr_features,
    "mi": mi_features,
    "rfe": rfe_features
}

In [181]:
X_train_all = X_train[all_features] # all features
X_train_boruta = X_train[boruta_features].copy() # boruta selection
X_train_corr = X_train[corr_features].copy() # cmap correlation
X_train_mi = X_train[mi_features].copy() # correlation, mi & clustering
X_train_rfe = X_train[rfe_features].copy() # rfecv

data = {
    "Method": [
        "All features",
        "Boruta",
        "Corelation",
        "Mutual Information",
        "RFE"
    ],
    "Feature quantity": [
        len(all_features),
        len(boruta_features),
        len(corr_features),
        len(mi_features),
        len(rfe_features)
    ],
    "Feature name": [
        ", ".join(all_features),
        ", ".join(boruta_features),
        ", ".join(corr_features),
        ", ".join(mi_features),
        ", ".join(rfe_features)
    ]
}

pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
df = pd.DataFrame(data)
df

,Method,Feature quantity,Feature name
0,All features,16,"age, hypertension, heart_disease, avg_glucose_level, bmi, gender_Male, gender_Other, ever_married_Yes, work_type_Never_worked, work_type_Private, work_type_Self-employed, work_type_children, Residence_type_Urban, smoking_status_formerly smoked, smoking_status_never smoked, smoking_status_smokes"
1,Boruta,3,"age, avg_glucose_level, bmi"
2,Corelation,8,"age, heart_disease, avg_glucose_level, hypertension, ever_married_Yes, smoking_status_formerly smoked, work_type_Self-employed, bmi"
3,Mutual Information,5,"age, hypertension, gender_Other, work_type_Private, smoking_status_formerly smoked"
4,RFE,15,"age, hypertension, heart_disease, avg_glucose_level, gender_Male, ever_married_Yes, work_type_Never_worked, work_type_Private, work_type_Self-employed, work_type_children, Residence_type_Urban, smoking_status_formerly smoked, smoking_status_never smoked, smoking_status_smokes, bmi"


# Ridge Hill Climbing

In [182]:
storage_url = "sqlite:///optuna_studies.db"
cv = StratifiedKFold(5, shuffle=True, random_state=42)

MODELS = [
    "logreg", "knn", "svm", "gnb", "dt",
    "rf", "ada", "gb", "extra",
    "lgbm", "xgb", "cat"
]

SMOTE_MODELS = {"logreg", "knn", "svm", "gnb"}

In [183]:
def load_all_model_predictions(
    MODELS,
    FEATURE_SETS,
    X_train,
    y_train,
    X_test
):
    """
    Safe loader for ensemble predictions.
    Handles:
    - old/new model artifacts
    - evolving feature sets
    - one-hot encoded categorical variables
    """

    train_preds = []
    test_preds = []
    model_names = []

    for feature_set_name, feature_list in FEATURE_SETS.items():

        for model_name in MODELS:

            model_path = f"models/{model_name}_{feature_set_name}_prob.pkl"

            with open(model_path, "rb") as f:
                artifact = pickle.load(f)

            model = artifact["model"]

            # --- JEDYNE ŹRÓDŁO PRAWDY O CECHACH ---
            if "features" in artifact:
                features = list(artifact["features"])

            elif hasattr(model, "feature_names_in_"):
                features = list(model.feature_names_in_)

            else:
                features = list(feature_list)

            # --- ALIGNMENT KOLUMN (KLUCZOWE) ---
            def align_features(X, features):
                X_aligned = X.copy()

                # dodaj brakujące kolumny
                missing_cols = set(features) - set(X_aligned.columns)
                for col in missing_cols:
                    X_aligned[col] = 0.0

                # usuń nadmiarowe kolumny + ustaw kolejność
                X_aligned = X_aligned[features]

                return X_aligned

            X_train_sel = align_features(X_train, features)
            X_test_sel = align_features(X_test, features)

            p_train = model.predict_proba(X_train_sel)[:, 1]
            p_test = model.predict_proba(X_test_sel)[:, 1]

            train_preds.append(p_train)
            test_preds.append(p_test)

            model_names.append(f"{model_name}_{feature_set_name}")

    P_train = np.column_stack(train_preds)
    P_test = np.column_stack(test_preds)

    return P_train, P_test, model_names

In [184]:
def ensemble_f1_score(P, y, weights, threshold, alpha=0.01):
    ensemble_proba = P @ weights
    y_pred = (ensemble_proba >= threshold).astype(int)

    f1 = f1_score(y, y_pred)
    ridge_penalty = alpha * np.sum(weights ** 2)

    return f1 - ridge_penalty


In [185]:
def hill_climbing_ensemble(
    P_train, y_train,
    step=0.02,
    max_iter=200,
    alpha=0.01
):
    n_models = P_train.shape[1]
    weights = np.ones(n_models) / n_models

    best_threshold, _ = find_best_f1_threshold_dummy(P_train, y_train, weights)
    best_score = ensemble_f1_score(P_train, y_train, weights, best_threshold, alpha)

    for _ in range(max_iter):
        improved = False

        for i in range(n_models):
            for delta in [+step, -step]:
                new_weights = weights.copy()
                new_weights[i] += delta

                if new_weights[i] < 0:
                    continue

                new_weights /= new_weights.sum()

                thr, _ = find_best_f1_threshold_dummy(
                    P_train, y_train, new_weights
                )

                score = ensemble_f1_score(
                    P_train, y_train, new_weights, thr, alpha
                )

                if score > best_score:
                    weights = new_weights
                    best_score = score
                    best_threshold = thr
                    improved = True

        if not improved:
            break

    return weights, best_threshold, best_score


In [186]:
def find_best_f1_threshold_dummy(P, y, weights, thresholds=np.linspace(0.01, 0.5, 50)):
    ensemble_proba = P @ weights

    best_f1 = 0.0
    best_thr = 0.5

    for t in thresholds:
        y_pred = (ensemble_proba >= t).astype(int)
        f1 = f1_score(y, y_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_thr = t

    return best_thr, best_f1


In [187]:
P_train, P_test, model_names = load_all_model_predictions(
    MODELS, FEATURE_SETS,
    X_train, y_train,
    X_test
)

weights, best_thr, train_score = hill_climbing_ensemble(
    P_train, y_train,
    step=0.02,
    alpha=0.01
)

y_test_pred = (P_test @ weights >= best_thr).astype(int)
f1_test = f1_score(y_test, y_test_pred)

print("ENSEMBLE RESULTS")
print("----------------")
print(f"F1 train: {train_score:.4f}")
print(f"F1 test : {f1_test:.4f}")
print(f"Threshold: {best_thr:.3f}")

for name, w in sorted(zip(model_names, weights), key=lambda x: -x[1]):
    if w > 0.01:
        print(f"{name:40s}  weight={w:.3f}")

os.makedirs("models/hc", exist_ok=True)

hc_path = "models/hc/hc_ensemble_v5.pkl"

hc_artifact = {
    "type": "hill_climbing_ensemble",
    "weights": weights,
    "threshold": best_thr,
    "model_names": model_names,
    "f1_train": train_score
}

with open(hc_path, "wb") as f:
    pickle.dump(hc_artifact, f)

ENSEMBLE RESULTS
----------------
F1 train: 0.8819
F1 test : 0.8791
Threshold: 0.490
knn_rfe                                   weight=0.081
knn_all                                   weight=0.057
lgbm_rfe                                  weight=0.038
svm_boruta                                weight=0.036
gb_rfe                                    weight=0.035
cat_rfe                                   weight=0.035
gb_correlation                            weight=0.032
knn_correlation                           weight=0.032
gb_all                                    weight=0.030
svm_all                                   weight=0.030
xgb_rfe                                   weight=0.028
ada_rfe                                   weight=0.025
xgb_mi                                    weight=0.024
ada_correlation                           weight=0.024
cat_boruta                                weight=0.023
dt_all                                    weight=0.023
knn_mi                             

In [188]:
import pickle
from pathlib import Path

def inspect_hc_ensemble(path, min_weight=0.01):
    with open(path, "rb") as f:
        artifact = pickle.load(f)

    print(f"\n📦 ENSEMBLE: {path}")
    print("-" * 60)

    print(f"F1 train : {artifact.get('f1_train', 'N/A')}")
    print(f"Threshold: {artifact.get('threshold')}")

    model_names = artifact["model_names"]
    weights = artifact["weights"]

    for name, w in sorted(zip(model_names, weights), key=lambda x: -x[1]):
        if w >= min_weight:
            print(f"{name:40s} weight={w:.4f}")

In [189]:
for p in Path("models/hc").glob("hc_ensemble*.pkl"):
    inspect_hc_ensemble(p)



📦 ENSEMBLE: models/hc/hc_ensemble.pkl
------------------------------------------------------------
F1 train : 0.8747367031666085
Threshold: 0.5
xgb_rfe_prob                             weight=0.0571
knn_rfe_prob                             weight=0.0518
gb_boruta_prob                           weight=0.0486
knn_all_prob                             weight=0.0463
xgb_correlation_prob                     weight=0.0364
gb_correlation_prob                      weight=0.0358
gb_rfe_prob                              weight=0.0352
gb_all_prob                              weight=0.0311
ada_correlation_prob                     weight=0.0275
extra_all_prob                           weight=0.0271
dt_all_prob                              weight=0.0260
ada_rfe_prob                             weight=0.0256
gb_mi_prob                               weight=0.0250
gnb_all_prob                             weight=0.0237
ada_all_prob                             weight=0.0231
svm_rfe_prob                  